In [13]:

# Collect Images For Face Recognition

import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    faces = face_classifier.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        x=x-10
        y=y-10
        cropped_face = img[y:y+h+50, x:x+w+50]

    return cropped_face


cap = cv2.VideoCapture(0)
count = 0
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (400, 400))
        #face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = './Images/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 200: 
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")



<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-13-5aa6a3e6c12c>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found


In [16]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/Train'
valid_path = 'Datasets/Test'

# add preprocessing layer to the front of VGG
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False
  
folders = glob('Datasets/Train/*')
x = Flatten()(vgg.output)


prediction = Dense(len(folders), activation='softmax')(x)


model = Model(inputs=vgg.input, outputs=prediction)

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                   zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                  target_size = (224, 224),
                                                  batch_size = 32,
                                                  class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Datasets/Test',
                                             target_size = (224, 224),
                                             batch_size = 32,
                                             class_mode = 'categorical')

r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

from keras.models import load_model

model.save('facefeatures_new_model.h5')



Found 400 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


<ipython-input-16-b72e1b4a4ce2>:59: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
13/13 [==============================] - 190s 15s/step - loss: 0.9613 - accuracy: 0.7275 - val_loss: 0.1562 - val_accuracy: 0.9594
Epoch 2/5
13/13 [==============================] - 161s 13s/step - loss: 0.0342 - accuracy: 0.9900 - val_loss: 0.1253 - val_accuracy: 0.9569
Epoch 3/5
13/13 [==============================] - 162s 13s/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.2113 - val_accuracy: 0.9264
Epoch 4/5
13/13 [==============================] - 168s 13s/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.1797 - val_accuracy: 0.9391
Epoch 5/5
13/13 [==============================] - 168s 13s/step - loss: 8.0179e-04 - accuracy: 1.0000 - val_loss: 0.1684 - val_accuracy: 0.9492


In [17]:

from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np

from keras.preprocessing import image
model = load_model('facefeatures_new_model.h5')

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Doing some Face Recognition with the webcam
video_capture = cv2.VideoCapture(0)
while True:
    _, frame = video_capture.read()
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
        img_array = np.array(im)
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        print(pred)
                     
        name="None matching"
        
        if(pred[0][0]>0.5):
            name='Manmeet'
           # cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
        cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

<>:19: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:19: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-17-2b7901096d2e>:19: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


[[1.0000000e+00 1.0122636e-13 7.7870415e-20 1.3317031e-12]]
[[1.0000000e+00 8.9826801e-13 1.5536190e-13 7.8954504e-10]]
[[7.3212492e-01 9.7184092e-09 2.6183860e-12 2.6787513e-01]]
[[9.9981433e-01 9.5703778e-10 1.3227876e-10 1.8573440e-04]]
[[9.99999762e-01 1.75772811e-07 8.48984008e-11 1.11710726e-07]]
[[1.0000000e+00 2.7186737e-09 1.2958010e-14 2.3063129e-10]]
[[1.0000000e+00 1.8889035e-10 1.3976739e-13 1.5150132e-08]]
[[1.0000000e+00 1.2171334e-12 4.2665080e-27 3.3155250e-15]]
[[1.0000000e+00 1.9779934e-08 3.3407490e-24 1.5340230e-13]]
[[1.0000000e+00 4.7853201e-09 4.4053586e-24 2.2886214e-11]]
[[1.0000000e+00 2.0238180e-09 2.1020767e-26 3.3944536e-12]]
[[1.0000000e+00 3.3384425e-13 1.0053461e-32 1.2060232e-15]]
[[1.0000000e+00 1.5120285e-15 1.5582803e-33 4.3765184e-15]]
[[1.0000000e+00 9.5467945e-17 3.5514413e-29 4.1794452e-17]]
[[9.9997663e-01 2.8278349e-09 8.0113978e-19 2.3330411e-05]]
[[1.0000000e+00 6.7799981e-11 3.0887263e-21 5.5738174e-12]]
[[1.0000000e+00 1.1384785e-10 3.8280